<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/ReAct_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [1]:
%pip install cohere tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 891.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.


In [3]:
%pip install openai httpx --upgrade
# %pip list

In [4]:
# Get the openai secret key:
import getpass
from openai import OpenAI
import re
import httpx

# Used to securely store your API key
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OpenAI_KEY')


In [16]:
# This code is adapted from Simon Willison https://til.simonwillison.net/llms/python-react-pattern
client = OpenAI(api_key=OPENAI_API_KEY)
#client.api_key = OPENAI_API_KEY

class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages)
        # print(completion.usage)
        return completion.choices[0].message.content

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()


action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(f"\n---- Printing result:{i}: {result}")

        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        print(f"\n---- Printing actions: {actions}")

        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

In [17]:
query("What is the capital of england?")


---- Printing result:1: Thought: I should look up England on Wikipedia to find out its capital.
Action: wikipedia: England
PAUSE

---- Printing actions: [<re.Match object; span=(0, 26), match='Action: wikipedia: England'>]
 -- running wikipedia England
Observation: <span class="searchmatch">England</span> is a country that is part of the United Kingdom. It shares land borders with Wales to its west and Scotland to its north, while Ireland is located

---- Printing result:2: Answer: The capital of England is not specified in the observation from Wikipedia. However, the capital of the United Kingdom, of which England is a part, is London.

---- Printing actions: []


In [12]:
query("What is the capital of england?")

Printing result: Thought: I should look up England on Wikipedia
Action: wikipedia: England
PAUSE
 -- running wikipedia England
Observation: <span class="searchmatch">England</span> is a country that is part of the United Kingdom. It shares land borders with Wales to its west and Scotland to its north, while Ireland is located
Printing result: Answer: The capital of England is not mentioned in the observation. Let me perform another action to find the answer.
Action: wikipedia: Capital of England
PAUSE
 -- running wikipedia Capital of England
Observation: London is the <span class="searchmatch">capital</span> and largest city <span class="searchmatch">of</span> <span class="searchmatch">England</span>, and the United Kingdom, with a population <span class="searchmatch">of</span> around 8.8 million, and the largest city in Western Europe
Printing result: Answer: The capital of England is London.


In [9]:
query("What is 2 + 2?")

Thought: This is a simple arithmetic calculation that I can solve easily.

Action: calculate: 2 + 2

PAUSE
 -- running calculate 2 + 2
Observation: 4
Answer: The sum of 2 + 2 is 4.
